# **Introduction**
## **Bias Mitigation in Facial Emotion Recognition Models**


###  Import Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import requests
from PIL import Image
from io import BytesIO
import torch
from torchvision import models

### Load and Prepare the Dataset

load the RAF-DB dataset and organize it into train and test sets, also preprocess the images to ensure they are in the correct format for model training.


In [ ]:
!pip install kaggle
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"helllojkqhjwe","key":"2fce3ca081f6ceffa5042a386ed326bd"}'}

In [ ]:
import os

os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')

!chmod 600 /root/.kaggle/kaggle.json

### Download RAF-DB Dataset



In [ ]:
!kaggle datasets download -d shuvoalok/raf-db-dataset
!unzip raf-db-dataset.zip -d /content/raf-db


### Load and Preprocess the Dataset
After downloading and extracting the dataset, load the images.

In [ ]:
train_dir = '/content/raf-db/DATASET/train'
test_dir = '/content/raf-db/DATASET/test'

train_emotions = os.listdir(train_dir)
test_emotions = os.listdir(test_dir)

def load_images_from_directory(directory, emotions):
    images = []
    labels = []
    for emotion in emotions:
        emotion_folder = os.path.join(directory, emotion)
        for img_name in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, img_name)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            images.append(img_array)
            labels.append(int(emotion))
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_directory(train_dir, train_emotions)
X_test, y_test = load_images_from_directory(test_dir, test_emotions)

### Load FairFace Dataset for Race Labels

load the FairFace dataset, which contains images with race annotations. We will match the face embeddings from FairFace to those from the RAF-DB dataset to assign race labels.


***Clone and Prepare FairFace Dataset***

In [ ]:
!pip install gdown

!gdown --id 1kXdAsqT8YiNYIMm8p5vQUvNFwhBbT4vQ -O res34_fair_align_multi_7_20190809.pt
!gdown --id 113QMzQzkBDmYMs9LwzvD-jxEZdBQ5J4X -O res34_fair_align_multi_4_20190809.pt

print("Models downloaded successfully!")


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1kXdAsqT8YiNYIMm8p5vQUvNFwhBbT4vQ
To: /content/res34_fair_align_multi_7_20190809.pt
100% 85.3M/85.3M [00:01<00:00, 60.4MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=113QMzQzkBDmYMs9LwzvD-jxEZdBQ5J4X
To: /content/res34_fair_align_multi_4_20190809.pt
100% 85.3M/85.3M [00:00<00:00, 181MB/s]
Models downloaded successfully!


In [ ]:
pip install --upgrade sympy


***Load FairFace Data***

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import torch
from torchvision import models
from sklearn.preprocessing import LabelEncoder

detected_faces_urls = [
    'https://raw.githubusercontent.com/dchen236/FairFace/master/detected_faces/race_Asian_face0.jpg',
    'https://raw.githubusercontent.com/dchen236/FairFace/master/detected_faces/race_Black_face0.jpg',
    'https://raw.githubusercontent.com/dchen236/FairFace/master/detected_faces/race_Latino_face0.jpg',
    'https://raw.githubusercontent.com/dchen236/FairFace/master/detected_faces/race_Middle_Eastern_face0.jpg',
    'https://raw.githubusercontent.com/dchen236/FairFace/master/detected_faces/race_White_face0.jpg'
]

def load_fairface_data(image_urls):
    fairface_images = []
    race_labels = []

    for img_url in image_urls:
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))
        img = img.resize((64, 64))
        img_array = np.array(img) / 255.0
        fairface_images.append(img_array)

        race_label = img_url.split('_')[1]
        race_labels.append(race_label)

    return np.array(fairface_images), np.array(race_labels)

X_fairface, y_fairface = load_fairface_data(detected_faces_urls)

label_encoder = LabelEncoder()
y_fairface_encoded = label_encoder.fit_transform(y_fairface)

model_7 = models.resnet34(pretrained=False, num_classes=7)
model_4 = models.resnet34(pretrained=False, num_classes=4)

pretrained_dict_7 = torch.load('res34_fair_align_multi_7_20190809.pt')
pretrained_dict_4 = torch.load('res34_fair_align_multi_4_20190809.pt')

pretrained_dict_7.pop('fc.weight', None)
pretrained_dict_7.pop('fc.bias', None)

pretrained_dict_4.pop('fc.weight', None)
pretrained_dict_4.pop('fc.bias', None)

model_7.load_state_dict(pretrained_dict_7, strict=False)
model_4.load_state_dict(pretrained_dict_4, strict=False)

model_7.fc = torch.nn.Linear(model_7.fc.in_features, 7)
model_4.fc = torch.nn.Linear(model_4.fc.in_features, 4)

model_7.eval()
model_4.eval()

print("Models loaded successfully!")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-9-2c962edd80eb>:48: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

Models loaded successfully!


**### Extract Embeddings for RAF-DB and FairFace**

In [ ]:
def extract_fairface_embeddings(fairface_images, model):
    fairface_embeddings = []
    for img in fairface_images:
        img_tensor = torch.tensor(img).float().unsqueeze(0).permute(0, 3, 1, 2)
        with torch.no_grad():
            embedding = model(img_tensor)
        fairface_embeddings.append(embedding.numpy())
    return np.array(fairface_embeddings)

fairface_embeddings = extract_fairface_embeddings(X_fairface, model_7)

def extract_raf_embeddings(raf_images):
    raf_embeddings = []
    for img in raf_images:
        img_tensor = torch.tensor(img).float().unsqueeze(0).permute(0, 3, 1, 2)
        with torch.no_grad():
            embedding = model_7(img_tensor)
        raf_embeddings.append(embedding.numpy())
    return np.array(raf_embeddings)

raf_embeddings_train = extract_raf_embeddings(X_train)
raf_embeddings_test = extract_raf_embeddings(X_test)

**### Assign Race Labels to RAF-DB Images Using Cosine Similarity**

In [ ]:
def assign_race_label(raf_embedding, fairface_embeddings, fairface_labels):
    raf_embedding_flat = raf_embedding.flatten().reshape(1, -1)
    fairface_embeddings_flat = fairface_embeddings.reshape(fairface_embeddings.shape[0], -1)

    similarities = cosine_similarity(raf_embedding_flat, fairface_embeddings_flat)
    closest_index = np.argmax(similarities)
    return fairface_labels[closest_index]

raf_race_labels_train = {}
for i, embedding in enumerate(raf_embeddings_train):
    race_label = assign_race_label(embedding, fairface_embeddings, y_fairface)
    raf_race_labels_train[i] = race_label

raf_race_labels_test = {}
for i, embedding in enumerate(raf_embeddings_test):
    race_label = assign_race_label(embedding, fairface_embeddings, y_fairface)
    raf_race_labels_test[i] = race_label

**### Save the Results**

In [ ]:
np.save("/content/raf_race_labels_train.npy", raf_race_labels_train)
np.save("/content/raf_race_labels_test.npy", raf_race_labels_test)

print("Training Data Race Labels:", raf_race_labels_train)
print("Test Data Race Labels:", raf_race_labels_test)

Training Data Race Labels: {0: 'faces/race', 1: 'faces/race', 2: 'faces/race', 3: 'faces/race', 4: 'faces/race', 5: 'faces/race', 6: 'faces/race', 7: 'faces/race', 8: 'faces/race', 9: 'faces/race', 10: 'faces/race', 11: 'faces/race', 12: 'faces/race', 13: 'faces/race', 14: 'faces/race', 15: 'faces/race', 16: 'faces/race', 17: 'faces/race', 18: 'faces/race', 19: 'faces/race', 20: 'faces/race', 21: 'faces/race', 22: 'faces/race', 23: 'faces/race', 24: 'faces/race', 25: 'faces/race', 26: 'faces/race', 27: 'faces/race', 28: 'faces/race', 29: 'faces/race', 30: 'faces/race', 31: 'faces/race', 32: 'faces/race', 33: 'faces/race', 34: 'faces/race', 35: 'faces/race', 36: 'faces/race', 37: 'faces/race', 38: 'faces/race', 39: 'faces/race', 40: 'faces/race', 41: 'faces/race', 42: 'faces/race', 43: 'faces/race', 44: 'faces/race', 45: 'faces/race', 46: 'faces/race', 47: 'faces/race', 48: 'faces/race', 49: 'faces/race', 50: 'faces/race', 51: 'faces/race', 52: 'faces/race', 53: 'faces/race', 54: 'faces